In [8]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Приведення розмірності до (32, 32, 3)
x_train_resized = np.repeat(x_train[..., np.newaxis], 3, -1)
x_train_resized = tf.image.resize(x_train_resized, [32, 32])
x_test_resized = np.repeat(x_test[..., np.newaxis], 3, -1)
x_test_resized = tf.image.resize(x_test_resized, [32, 32])

# Нормалізація даних
x_train_resized = x_train_resized / 255.0
x_test_resized = x_test_resized / 255.0


# Створення моделі:

In [9]:
# Завантаження VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Заморожуємо шари VGG16
for layer in base_model.layers:
    layer.trainable = False

# Додавання власних шарів
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(10, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)


# Компіляція моделі:

In [10]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(x_train_resized, y_train,
                    validation_data=(x_test_resized, y_test),
                    epochs=10, batch_size=128)


Epoch 1/10
469/469 [==============================] - 118s 250ms/step - loss: 1.1285 - accuracy: 0.6233 - val_loss: 0.6893 - val_accuracy: 0.7726
Epoch 2/10
469/469 [==============================] - 121s 258ms/step - loss: 0.6684 - accuracy: 0.7688 - val_loss: 0.5805 - val_accuracy: 0.8011
Epoch 3/10
469/469 [==============================] - 121s 258ms/step - loss: 0.5835 - accuracy: 0.7944 - val_loss: 0.5304 - val_accuracy: 0.8155
Epoch 4/10
469/469 [==============================] - 124s 264ms/step - loss: 0.5393 - accuracy: 0.8095 - val_loss: 0.5003 - val_accuracy: 0.8223
Epoch 5/10
469/469 [==============================] - 124s 264ms/step - loss: 0.5090 - accuracy: 0.8210 - val_loss: 0.4797 - val_accuracy: 0.8299
Epoch 6/10
469/469 [==============================] - 127s 271ms/step - loss: 0.4874 - accuracy: 0.8261 - val_loss: 0.4620 - val_accuracy: 0.8356
Epoch 7/10
469/469 [==============================] - 125s 266ms/step - loss: 0.4691 - accuracy: 0.8335 - val_loss: 0.4500 -

# Оцінка моделі:

In [11]:
loss, accuracy = model.evaluate(x_test_resized, y_test)
print(f"Test accuracy: {accuracy}")

313/313 [==============================] - 22s 69ms/step - loss: 0.4239 - accuracy: 0.8482
Test accuracy: 0.8482000231742859


# Висновок:

При використанні передтренованої моделі VGG16 на датасеті fasion_mnist ми досягли точності близько 84.82% на тестовому наборі даних. Це свідчить про ефективність використання VGG16 як згорткової основи для видобутку ознак у задачі класифікації для цього конкретного набору даних. Використання передтренованих мереж, як VGG16, може значно підвищити продуктивність і зменшити час навчання, особливо коли у нас є обмежені обчислювальні ресурси або невеликі набори даних.